In [124]:
import os
import csv
import sys
import string

csv.field_size_limit(sys.maxsize)

9223372036854775807

###1. Load data: read in csv as a list of dictionaries

In [125]:
protests = []

# open file
with open('protests.csv', 'rU') as csvfile: 
    
    # create a reader
    reader = csv.DictReader(csvfile) 
    
    # loop through rows
    for row in reader: 
        
        # append each row to the list
        protests.append(row) 

In [43]:
type(protests)

list

In [129]:
keys = protests[1].keys()
keys

['', 'PUBLICATION', 'TITLE', 'COUNTRY', 'LENGTH', 'TEXT', 'DATE']

###3. Write function that passes two arguments: article and city
Function will search the text of the article for the city of interest, identify its index, and then search for the root words "protest" or its synonym "manifest" (short for manifestação, and permutations thereof) within 30 words of the city name.

#####Note: 
Previous trial-and-error of earlier versions of this function revealed that articles in which there was any punctuation attached to the city name, the function would not detect it as one of the "search terms." In other words, an article that said "manifestantes protestaram em Curitiba, mas estavam pacíficos" would not detect "protest" near "Curitiba" because of the comma. The same was true with compound city names. Hence the effort to modify the function to tokenize the articles at the beginning, remove punctuation, and concatenate compound names.

These were the special cases I needed to account for:

| Condition       | Example          | Worked?  |
|------------- |-------------| -----|
| accent      | "Brasília" | yes |
| punctuation      | "Rio."      | no    |
| compound | "São Paulo"      |    no |

In [110]:
def query_cities(article, city):
    
    #this removes punctuation
    article = article.translate(None, string.punctuation)
    
    #this removes endline characters
    article = article.translate(None, "\n")
    
    #this concatenates compound names with an underscore
    article = article.replace("São Paulo", "São_Paulo")
    article = article.replace("Belo Horizonte", "Belo_Horizonte")
    article = article.replace("Porto Alegre", "Porto_Alegre")
    article = article.replace("Rio de Janeiro", "Rio")
    
    text = article.split()
    if city in text:
        takeIndex = text.index(city)
        wc = len(text)
        if takeIndex > 30 and takeIndex < wc - 30:
            subset = ' '.join(text[takeIndex-30:takeIndex+30])
        elif takeIndex < 30 and takeIndex < wc - 30:
            subset = ' '.join(text[:takeIndex+30])
        elif takeIndex > 30 and takeIndex > wc - 30:
            subset = ' '.join(text[takeIndex-30:])

        if 'protest'or 'manifest' in subset:
            return(1)
    else:
        return(0)

###5. Call function on each of the 10 cities
This is a nested loop that loops over all of the articles ('TEXT') from inside of the dictionaries in protest. For each article, loop over all of the 10 cities I want to search for. Each time, it adds an key to the dictionary that contains the results from that search. 

In [120]:
to_search_for = ["Rio", "Brasília", "Belo_Horizonte", "Curitiba", "Salvador",\
                 "São_Paulo", "Porto_Alegre", "Fortaleza", "Recife", "Campinas"]

for i in protests:
    for j in to_search_for:
        i[j] = query_cities(i['TEXT'], j)

###7. Write CSV

In [122]:
keys = protests[0].keys()
print keys

['', 'PUBLICATION', 'Salvador', 'TITLE', 'COUNTRY', 'Belo_Horizonte', 'Campinas', 'Bras\xc3\xadlia', 'Rio', 'LENGTH', 'Fortaleza', 'S\xc3\xa3o_Paulo', 'TEXT', 'DATE', 'Curitiba', 'Recife', 'Porto_Alegre']


In [123]:
#writing the rest
with open('protest_cities.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(protests)